In [31]:
import KiMoPack


In [32]:
import os,sys
import pandas as pd
import numpy as np
import matplotlib,lmfit
import matplotlib.pyplot as plt
try:
    import KiMoPack.plot_func as pf
except:
    print("General installation did not work, try to import from the same folder as,a workaround")
    import plot_func as pf


In [33]:
ta=np.loadtxt('TA_2D-DATA.mx3')



In [34]:
%matplotlib qt

In [10]:
ta.Plot_RAW(title='Kinetic traces at selected probe wavelengths', plotting=[1])

AttributeError: 'numpy.ndarray' object has no attribute 'Plot_RAW'

In [35]:
ta=pf.TA('gui')

In [20]:
ta.Plot_RAW(title='Kinetic traces at selected probe wavelengths', plotting=[0])

In [37]:
ta.Plot_RAW(title='Kinetic traces at selected probe wavelengths', plotting=[1])
ta.Plot_RAW(title='TA spectra at selected delay-times', plotting=[2])
ta.Plot_RAW(title='2D-Plot', plotting=[0])

In [38]:
ta.Plot_RAW(title='SVD', plotting=[3], savetype='pdf')

In [23]:
# Define fit parameters
ta.par=lmfit.Parameters()
# rate constants
ta.par.add('k0',value=1/2, min=1/10.0, max=1/0.25, vary=True)
ta.par.add('k1',value=1/150, min=1/200.0, max=1/10.0, vary=True)
ta.par.add('k2',value=1/100000, vary=False)
# time-zero parameter fixed during fit
ta.par.add('t0',value=0.0, min=-0.1, max=0.1, vary=False)
# pump-pulse width parameter fixed during fit
ta.par.add('resolution', value=0.12, min=0.04, max=0.20, vary=False)
# Select a in-build model (here: independent parallel decay)
ta.mod='exponential'
# set delay-time range for fit
ta.timelimits=[0.35,2000]
# global lifetime analysis (kinetic modeling)
ta.Fit_Global()



Fit Results:
Model Used: exponential

The minimum error is:1.12443969e-03
The minimum R2-value is:9.67552114e-01

In Rates

               value init_value   vary    min  max  expr
k0          0.392438        0.5   True    0.1  4.0  None
k1             0.005   0.006667   True  0.005  0.1  None
k2           0.00001    0.00001  False      0  inf  None
t0               0.0        0.0  False   -0.1  0.1  None
resolution      0.12       0.12  False   0.04  0.2  None

The rates converted to times with unit ps

               value init_value   vary   min    max  expr
k0          2.548173        2.0   True  0.25   10.0  None
k1             200.0      150.0   True  10.0  200.0  None
k2          100000.0   100000.0  False   0.0    inf  None
t0               0.0        0.0  False  -0.1    0.1  None
resolution      0.12       0.12  False  0.04    0.2  None


In [24]:
# fit-error estimation in a confidence interval of 95%
ta.Fit_Global(confidence_level=0.95)


Trying to find k0, lower confidence limit
Trying to find k0, upper confidence limit
Trying to find k1, lower confidence limit
error in k1 at lower limit
Trying to find k1, upper confidence limit
it took 72 optimisations to get the confidence

Fit Results:
Model Used: exponential

The minimum error is:1.12443969e-03
The minimum R2-value is:9.67552114e-01

In Rates with confidence interval to level of 95.0

               value lower_limit upper_limit init_value   vary    min  max  expr
k0          0.392438    0.368153    0.417721        0.5   True    0.1  4.0  None
k1             0.005        None    0.007444   0.006667   True  0.005  0.1  None
k2           0.00001        None        None    0.00001  False      0  inf  None
t0               0.0        None        None        0.0  False   -0.1  0.1  None
resolution      0.12        None        None       0.12  False   0.04  0.2  None

The rates converted to times with unit ps with confidence interval to level of 95.0

               valu

In [29]:
ta_listen=[ta.Copy(),ta.Copy()] #create a list for comparision
#the Filename can be manipulated to use the automatic naming
ta_listen[0].filename="upper confidence limit"
ta_listen[1].filename="lower confidence limit"
for i in range(2):
    #short name for the calculated results for reduced writing
    par=ta.re['fit_results_rates'].copy()
    if i == 0:
        #overwrite the value with the limits
        par.loc['k0','value']=par.loc['k0','upper_limit']
    else:
        par.loc['k0','value']=par.loc['k0','lower_limit']
    # Write the fit results as input parameter
    ta_listen[i].par=pf.pardf_to_par(par)
    for key in ta_listen[i].par.keys():
        # Lock the parameter so that only the spectra are calculated
        ta_listen[i].par[key].vary=False
    # Run the global fit to calculate the new spectra
    ta_listen[i].Fit_Global()


ATTENTION: we have not optimized anything but just returned the parameters

Fit Results:
Model Used: exponential

The minimum error is:1.12502960e-03
The minimum R2-value is:9.67535091e-01

In Rates

               value init_value   vary    min  max  expr
k0          0.417721   0.417721  False    0.1  4.0  None
k1             0.005      0.005  False  0.005  0.1  None
k2           0.00001    0.00001  False      0  inf  None
t0               0.0        0.0  False   -0.1  0.1  None
resolution      0.12       0.12  False   0.04  0.2  None

The rates converted to times with unit ps

               value init_value   vary   min    max  expr
k0          2.393941   2.393941  False  0.25   10.0  None
k1             200.0      200.0  False  10.0  200.0  None
k2          100000.0   100000.0  False   0.0    inf  None
t0               0.0        0.0  False  -0.1    0.1  None
resolution      0.12       0.12  False  0.04    0.2  None
ATTENTION: we have not optimized anything but just returned the pa

In [30]:
%matplotlib qt
ta.Compare_at_wave(fitted=True,
other=ta_listen,
rel_wave=[450,590],
width=50,
linewidth=3)
ax=plt.gca() # get a handle to the drawn axis
ax.set_ylim(-50e-3,20e-3) # set the ylimit of the plot


(-0.05, 0.02)